In [71]:
import pandas as pd
from matplotlib import pyplot as plt

plt.style.use("fivethirtyeight")
%matplotlib inline

In [72]:
df = pd.read_csv('historical_data.csv')

In [73]:
#dropping some extreme cases that skew the data
df = df.drop([1, 2950, 3904, 1032, 1926, 2620, 589, 2947, 948, 1031, 1353])

In [74]:
#cleaning the dates
df[['start_timestamp','stop_timestamp']] = df[['start_timestamp','stop_timestamp']].replace('T', ' ', regex=True)
df[['start_timestamp','stop_timestamp']] = df[['start_timestamp','stop_timestamp']].replace('Z', '', regex=True) 
df[['start_timestamp','stop_timestamp']] = df[['start_timestamp','stop_timestamp']].apply(pd.to_datetime)

In [75]:
#calculating the time of charge session in minutes
df['tot_minutes_diff'] = (df.stop_timestamp - df.start_timestamp) / pd.Timedelta(minutes=1)

In [76]:
#Average time of a charging session
df.tot_minutes_diff.mean()

39.4853595232758

In [77]:
df['weekday'] = df['start_timestamp'].dt.dayofweek

In [78]:
#Average time of a charging session per day of the week
df.groupby('weekday',as_index=False)['tot_minutes_diff'].mean()

,weekday,tot_minutes_diff
0,0,41.020629
1,1,34.183215
2,2,48.874278
3,3,35.855196
4,4,41.377929
5,5,41.139113
6,6,36.785290


In [79]:
#Average time of a charging session per charge station
df_seperated=df.groupby('charge_box_id', as_index=False)['tot_minutes_diff'].mean()
df_tags=df.groupby('charge_box_id', as_index=False)['id_tag'].first()
df_seperated['id_tag']=df_tags['id_tag']
df_seperated

,charge_box_id,tot_minutes_diff,id_tag
0,17111799,47.593058,LeopolisHotelLviv1
1,17516081,10.167408,PaintAndGo
2,17538670,42.106907,predefinedTokenId
3,17968671,64.447640,futura2
4,17968674,95.693278,PodolianyShopping1
5,17968677,154.116811,BoschFormula1
6,17968679,11.398161,ConceptAria
7,17968680,60.184817,SoftServeLviv1
8,17968681,56.690280,vash_dim1
9,19028818,27.627872,RenaultLviv


In [80]:
df['energy transferred(kWh)'] = (df['stop_value'] - df['start_value'])/1000

In [81]:
df['Power(kW)'] = df['energy transferred(kWh)']/(df['tot_minutes_diff']/60)

In [82]:
df = df.assign(time_of_Day=pd.cut(df.start_timestamp.dt.hour,[-1, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23],
                           labels=['0-2am', '2-4am', '4-6am', '6-8am', '8-10am', '10-12am', '12-2pm',
                                   '2-4pm', '4-6pm', '6-8pm', '8-10pm', '10-12pm']))

In [83]:
df

,sessionID,charge_box_id,connector_id,id_tag,start_timestamp,start_value,stop_timestamp,stop_value,stop_reason,location_id,tot_minutes_diff,weekday,energy transferred(kWh),Power(kW),time_of_Day
0,4633,t54_hu1_0820_003,3,tag_003_2,2020-11-12 12:44:02.000,3669798,2020-11-12 12:58:52.000,3670994.0,NaN,370377.0,14.833333,3,1.196,4.837753,12-2pm
2,4631,19028824,1,predefinedTokenId,2020-11-12 12:01:25.742,3787009,2020-11-12 12:01:30.554,3787009.0,DeAuthorized,370377.0,0.080200,3,0.000,0.000000,12-2pm
3,4630,19926067,1,Lumier,2020-11-12 11:39:55.987,2857299,2020-11-12 12:36:03.782,2863492.0,EVDisconnected,370377.0,56.129917,3,6.193,6.619999,10-12am
4,4629,t54_hu1_0820_002,3,tag_002_2,2020-11-12 11:09:23.000,2694321,2020-11-12 11:39:12.000,2700103.0,NaN,370377.0,29.816667,3,5.782,11.635103,10-12am
5,4628,t54_hu1_0820_003,1,tag_003_1,2020-11-12 10:55:59.000,855793,2020-11-12 11:56:01.000,865834.0,NaN,370377.0,60.033333,3,10.041,10.035425,10-12am
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4467,58,t54_hu1_0820_003,1,tag_003_1,2020-07-30 18:24:11.000,5465,2020-07-30 18:26:25.000,5611.0,NaN,NaN,2.233333,3,0.146,3.922388,6-8pm
4468,57,t54_hu1_0820_003,1,tag_003_1,2020-07-30 18:09:21.000,5351,2020-07-30 18:11:06.000,5462.0,NaN,NaN,1.750000,3,0.111,3.805714,6-8pm
4469,56,t54_hu1_0820_003,1,tag_003_1,2020-07-30 17:57:43.000,4674,2020-07-30 18:07:33.000,5349.0,NaN,NaN,9.833333,3,0.675,4.118644,4-6pm
4470,55,t54_hu1_0820_003,1,tag_003_1,2020-07-30 17:53:56.000,4507,2020-07-30 17:56:26.000,4670.0,NaN,NaN,2.500000,3,0.163,3.912000,4-6pm


In [84]:
#histogram for a day(average minutes spent charging)
histogram_df1 = df.groupby('time_of_Day', as_index=False)['tot_minutes_diff'].mean()
histogram_df1


,time_of_Day,tot_minutes_diff
0,0-2am,36.514919
1,2-4am,41.436411
2,4-6am,48.064626
3,6-8am,61.472002
4,8-10am,42.074762
5,10-12am,42.364363
6,12-2pm,38.431084
7,2-4pm,48.748405
8,4-6pm,37.286459
9,6-8pm,30.050934


In [85]:
#histogram for a day(total energy demand)
histogram_df2 = df.groupby('time_of_Day', as_index=False)['energy transferred(kWh)'].sum()
histogram_df2

,time_of_Day,energy transferred(kWh)
0,0-2am,809.802
1,2-4am,695.827
2,4-6am,1030.010
3,6-8am,2713.912
4,8-10am,3378.568
5,10-12am,4130.731
6,12-2pm,3927.167
7,2-4pm,4432.230
8,4-6pm,4201.966
9,6-8pm,2949.694


In [86]:
charger_power_df = df.groupby('charge_box_id', as_index=False)['Power(kW)'].mean()
charger_power_df['id_tag']=df_tags['id_tag']
charger_power_df.sort_values(by=['Power(kW)'], inplace=True)
charger_power_df

,charge_box_id,Power(kW),id_tag
16,19657841,0.705163,ste
10,19028819,1.151587,Kazka
22,EVB-P20031536,1.258136,EvBoxTest
13,19028824,2.718556,predefinedTokenId
15,19459689,2.802261,predefinedTokenId
5,17968677,3.024928,BoschFormula1
20,19926068,3.145148,cgtn_karpaty
7,17968680,3.216094,SoftServeLviv1
17,19707524,3.761453,VillaMitades
8,17968681,4.710770,vash_dim1
